In [1]:
import numpy as np
import json
import os

In [2]:
# read multiplication table file
infile = [np.asarray(json.loads(line)) for line in open(os.path.abspath(os.pardir) + '\z2_s4_001.json', 'r')]
mixer_index = infile[0]
M = infile[1] #multipliers
T = infile[2] #multiplication table

assert T.shape[0] == T.shape[1] == T.shape[2]
len_T = T.shape[0]
print("Number of conjugacy classes: {}" .format(len_T))


assert M.shape[1] == len_T
len_M = M.shape[0]
print("Number of multipliers: {}" .format(len_M))

#print("Multiplier Indices: {}" .format(M))

Number of conjugacy classes: 33
Number of multipliers: 9


In [95]:
def encode_msg(m, len_T, nbytes = 2):
    chars_per_item = nbytes * 2
    #convert string to binary sequence
    bin_version = ''.join(format(ord(i), '08b') for i in m)
    #seperate binary sequence into list of 'chunks' of length 'chars_per_item'
    len_bin = len(bin_version)
    num_chunks = len(bin_version) / chars_per_item
    res = []
    for start in range(0, len(bin_version), chars_per_item):
        res.append(bin_version[start:start + chars_per_item])
    res = [int(e) for e in res]
    #seperate chunks into lists of length 'len_T'
    res = [res[i:i + len_T] for i in range(0, len(res), len_T)]
    # possibly, the final list will need to be padded
    # we choose Pad = 2 as it is not a binary number
    pad = 2
    pad_list = [pad]*len_T
    # we pad from the end of the list for efficacy
    pad_list[-len(res[-1]):] = res[-1]
    res[-1] = pad_list
    return [np.asarray(e) for e in res]

In [96]:
def brmult_list(T, L, sel = "none"):
    if sel == "none":
        pass
    else:
        L = L[sel,:]
        
    for l in L:
        if all(l == L[0]): # initialize product
            chi = l
        else:
            temp = np.zeros(len(l))
            for i in range(len(l)):
                for j in range(len(l)):
                    temp+= chi[i]*l[j]*T[i][j]
            chi = temp
            #print("-------------------------------------------------------------")
            #print(chi) 
    return chi

In [97]:
msg = "abcdefghijklmnopqrs"
msg_utf = encode_msg(msg, len_T)
print(msg_utf)

[array([ 110,    1,  110,   10,  110,   11,  110,  100,  110,  101,  110,
        110,  110,  111,  110, 1000,  110, 1001,  110, 1010,  110, 1011,
        110, 1100,  110, 1101,  110, 1110,  110, 1111,  111,    0,  111]), array([  2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,
         2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,
         2,   2,   1, 111,  10, 111,  11])]


In [103]:
bin_msg = ''.join(format(ord(i), '08b') for i in msg)
bin_msg
#''.join(chr(int(msg[i*8:i*8+8]),2) for i in range(len(msg)//8))

'01100001011000100110001101100100011001010110011001100111011010000110100101101010011010110110110001101101011011100110111101110000011100010111001001110011'

In [ ]:
def to_bytestr(m, nbytes =2):
    return ''.join(chr(int(m[i*8:i*8+8]),2) for i in range(len(m)//8))

In [98]:
multiplier = brmult_list(T,M)
encryption = [brmult_list(T,[e,multiplier]) for e in msg_utf]
#encryption = brmult_list(T,[msg_utf,multiplier])
print(encryption)

[array([ 2.222e+03, -1.333e+03,  3.654e+03, -1.010e+03, -2.442e+03,
       -2.343e+03, -2.210e+02, -2.432e+03, -2.210e+02, -1.533e+03,
       -1.452e+03, -1.541e+03, -2.532e+03,  2.553e+03, -2.443e+03,
        1.000e+03,  1.000e+00, -8.900e+02,  1.000e+00, -8.990e+02,
        1.000e+00,  1.232e+03,  3.310e+02,  1.321e+03,  1.000e+00,
        1.322e+03,  1.000e+00, -1.332e+03, -3.320e+02, -1.222e+03,
       -2.220e+02, -1.110e+02,  1.110e+02]), array([  15.,  -17.,   30.,   -4.,  -19.,  -19.,  -13.,  -18.,  -13.,
        -18.,  -18.,  -18.,  -18.,   28.,  -18.,    2.,    9.,    9.,
          9.,    9.,    9.,   14.,   14.,   14.,    9.,   14.,    9.,
        -24.,  -23., -122.,  -21., -122.,   11.])]


In [99]:
decryption = [brmult_list(T,[e,multiplier]).astype(int) for e in encryption]
print(decryption)


[array([ 110,    1,  110,   10,  110,   11,  110,  100,  110,  101,  110,
        110,  110,  111,  110, 1000,  110, 1001,  110, 1010,  110, 1011,
        110, 1100,  110, 1101,  110, 1110,  110, 1111,  111,    0,  111]), array([  2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,
         2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,
         2,   2,   1, 111,  10, 111,  11])]


In [93]:
def to_bytestr(m, nbytes =2):
    return ''.join(chr(m[i*8:i*8+8],2 for i in range(len(m)//8))

In [94]:
msg_rec = to_bytestr(decryption[0])

TypeError: chr() takes exactly one argument (2 given)